In [1]:
from numpy.random import seed
seed(1)
import sklearn
sklearn.__version__
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

import re
from sklearn.metrics import classification_report

In [2]:
data2017 = pd.read_csv("weather_data_[2017].csv", sep="\t")
data2018 = pd.read_csv("weather_data_[2018].csv", sep="\t")
data2019 = pd.read_csv("weather_data_[2019].csv", sep="\t")
data = pd.concat([data2017,data2018,data2019], ignore_index=True, axis=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8512 entries, 0 to 8511
Data columns (total 12 columns):
time             8512 non-null object
month            8512 non-null int64
temperature      8512 non-null float64
feelslike        8512 non-null float64
wind             8512 non-null float64
direction        8512 non-null object
gust             8512 non-null float64
cloud            8512 non-null float64
humidity         8512 non-null float64
precipitation    8512 non-null float64
pressure         8512 non-null float64
weather          8512 non-null object
dtypes: float64(8), int64(1), object(3)
memory usage: 798.1+ KB


In [3]:
# Xếp loại các loại thời tiết thành 2 dạng là có mưa và không mưa
list_norain = ['Clear', 'Cloudy', 'Mist', 'Sunny', 'Partly cloudy', 'Thundery outbreaks possible']
list_rain = ['Light drizzle','Light rain','Light rain shower', 'Patchy light drizzle', 'Patchy light rain', 
             'Patchy light rain with thunder','Patchy rain possible','Heavy rain','Heavy rain at times',
             'Moderate or heavy rain shower','Moderate rain', 'Moderate rain at times', 'Overcast','Torrential rain shower']

In [4]:
# Lấy ra cột y để train
# Cột y này là giá trị thời tiết được xếp loại
y = []
for w in data.weather:
    if w in list_norain:
        y+= ['norain']
    elif w in list_rain:
        y+= ['rain']
    else:
        y+= ['unknown']
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo
# nên chúng ta xóa dòng đầu tiên
y.pop(0)

'norain'

In [5]:
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo nhưng không có giá trị đó nên xóa dòng cuối cùng
x = data.drop(8511)

In [6]:
# Tách tập train và tập test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, stratify = y, random_state=0)

In [7]:
# Tách tập train và tập validation
x_train_v, x_val, y_train_v, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify = y_train, random_state=0)

In [8]:
# Góm nhóm các giá trị của các cột có kiểu category
# Cột month gom thành 2 giá trị gồm "mua" (5->11) và "kho" (12->4)
# Cột direction thì xóa bỏ kí tự đầu tiên nếu vượt quá 2 ký tự
# Cột weather thì gom nhóm giống như khi gom nhóm y
class ColStd(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df,y=None):
        df = X_df[['time',
                   'month',
                   'temperature',
                   'feelslike',
                   'wind',
                   'direction',
                   'gust',
                   'cloud',
                   'humidity',
                   'pressure',
                   'weather']]#khong lay cot precipitatinon
        temp_time =[]
        for x in df['time']:
            t = int(re.findall(r'(\d{1,2}):\d{2}',x)[0])
            if ((t>=6) & (t<=15)): 
                temp_time+=['sang']
            else:
                temp_time+=['toi']
        df.time=temp_time
        temp_month =[]
        for x in df['month']:
            if x in [5,6,7,8,9,10,11]:
                temp_month += ['mua'] 
            elif x in [12,1,2,3,4]:
                temp_month += ['kho']
            else:
                temp_month += ['kho']
        df.month = temp_month
        temp_dic = []
        for x in df['direction']:
            if len(x) == 3:
                temp_dic += [x[1:]]
            else:
                temp_dic += [x]
        df.direction = temp_dic
        temp_wea = []
        for w in df['weather']:
            if w in list_norain:
                temp_wea+= ['norain']
            #elif w in list_ltrain:
            #    temp_wea+= ['ltrain']
            elif w in list_rain:
                temp_wea+= ['rain']
            else:
                temp_wea+= ['unknown']
        df.weather = temp_wea
        return df

In [9]:
# Tạo các Pipeline để tiền xử lý các giá trị có trong dữ liệu
nume_cols = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'humidity', 'pressure']
#nume_cols_nan = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'humidity','precipitation', 'pressure']
cate_cols = ['time', 'month', 'direction', 'weather']

nume_trans = Pipeline(steps=[
    ('imputer1', SimpleImputer(missing_values = np.nan, strategy = 'mean')),
    ('imputer2', SimpleImputer(missing_values = 0, strategy='mean'))
])

cate_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy= 'most_frequent')),
    ('onehot', OneHotEncoder(dtype=int, handle_unknown= 'ignore'))
])

cols_trans = ColumnTransformer(
    transformers=[
        ('nume', nume_trans, nume_cols),
        ('cate', cate_trans, cate_cols)
])

preprocess_pipeline = Pipeline(steps=[
    ('colstd', ColStd()),
    ('coltrans', cols_trans),
    ('std', StandardScaler())
])

# Tiền xử lý tập train x
preprocessed_train_X = preprocess_pipeline.fit_transform(x_train_v)

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
# Tiền xử lý tập validation x
preprocess_val_X = preprocess_pipeline.transform(x_val)

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [11]:
from sklearn.utils.class_weight import compute_class_weight 

In [12]:
#khảo sát số lượng từng nhãn xem có cân bằng hay không
rain = 0
norain =0
for y in y_train:
    if y =='rain':
        rain+=1
    else:
        norain+=1
print(f"rain:{rain} norain{norain}")

rain:1883 norain5776


In [13]:
# Tạo ra full_pipeline bao gồm tiền xử lý và model
mlpclf_model = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('mlpclf', MLPClassifier(hidden_layer_sizes=(20),activation='tanh', solver='lbfgs', random_state=0, max_iter=500,))
])
#hidden_layer_sizes = (10, 20, 50, 100)

In [14]:
# train và tìm ra các tham số tốt nhất
train_errs = []
val_errs = []
iter = []
alphas = [0.001,0.01, 0.1, 1, 10,20, 100]

hidden_layer_sizes = [1,10,20,50,70,100]
best_val_err = float('inf'); best_alpha = None; best_hls = None;
for alpha in alphas:
    for hls in hidden_layer_sizes:
        iter +=[f"alpha = {alpha}, hls = {hls}"]
        print(iter[-1])
        mlpclf_model.set_params(mlpclf__alpha=alpha,mlpclf__hidden_layer_sizes=(hls))
        mlpclf_model.fit(x_train_v,y_train_v)
        train_err =100*(1 - mlpclf_model.score(x_train_v,y_train_v))
        val_err =100*(1 - mlpclf_model.score(x_val,y_val))

        train_errs += [train_err]
        val_errs +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_alpha = alpha
            best_hls = hls


alpha = 0.001, hls = 1


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

alpha = 0.001, hls = 10


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

alpha = 0.001, hls = 20


...

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


alpha = 100, hls = 70


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

alpha = 100, hls = 100


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [15]:
for i in range(len(train_errs)):
    print(f"{iter[i]} \n train_err :{train_errs[i]}\t val_err:{val_errs[i]}")
    print()
print(f"best alpha: {best_alpha}")
print(f"best hidden layer size: {best_hls}")

alpha = 0.001, hls = 1 
 train_err :17.153582503672272	 val_err:17.167101827676245

alpha = 0.001, hls = 10 
 train_err :13.677166639464666	 val_err:16.840731070496084

alpha = 0.001, hls = 20 
 train_err :10.429247592622815	 val_err:19.973890339425594

alpha = 0.001, hls = 50 
 train_err :4.112942712583645	 val_err:21.801566579634468

alpha = 0.001, hls = 70 
 train_err :0.04896360372123043	 val_err:21.148825065274146

alpha = 0.001, hls = 100 
 train_err :0.016321201240410144	 val_err:22.32375979112271

alpha = 0.01, hls = 1 
 train_err :17.12094010119145	 val_err:17.101827676240212

alpha = 0.01, hls = 10 
 train_err :13.448669822098902	 val_err:16.710182767624016

alpha = 0.01, hls = 20 
 train_err :10.67406561122899	 val_err:18.99477806788512

alpha = 0.01, hls = 50 
 train_err :4.194548718785707	 val_err:21.540469973890342

alpha = 0.01, hls = 70 
 train_err :0.08160600620206182	 val_err:20.430809399477813

alpha = 0.01, hls = 100 
 train_err :0.016321201240410144	 val_err:21.148

In [16]:
for i in range(len(train_errs)):
    if (((i+1)%6==0) & (i!=0)):
        print(f"{round(train_errs[i],7)}",end ="\n")
    else:
        print(f"{round(train_errs[i],7)}",end ="\t")

print()
for i in range(len(val_errs)):
    if (((i+1)%6==0 & (i!=0))):
        print(f"{round(val_errs[i],7)}",end ="\n")
    else:
        print(f"{round(val_errs[i],7)}",end ="\t")

17.1535825	13.6771666	10.4292476	4.1129427	0.0489636	0.0163212
17.1209401	13.4486698	10.6740656	4.1945487	0.081606	0.0163212
17.2025461	13.8403787	11.5227681	4.7984332	1.4852293	0.1142484
17.1046189	13.791415	12.0287253	7.1323649	5.1738208	4.4393667
16.9250857	14.6727599	13.3670638	13.1875306	13.1875306	13.220173
16.8761221	15.4235352	14.7217235	14.7706871	14.6890811	14.9828627
17.1046189	16.6965889	16.9414069	17.1862249	17.2025461	17.2351885

17.1671018	16.8407311	19.9738903	21.8015666	21.1488251	22.3237598
17.1018277	16.7101828	18.9947781	21.54047	20.4308094	21.1488251
17.2976501	16.5796345	18.4725849	20.4308094	21.2140992	21.4751958
17.4934726	16.5143603	16.5796345	19.4516971	18.7336815	19.386423
17.7545692	16.6449086	16.3185379	15.9921671	16.1879896	16.0574413
17.8198433	16.3185379	16.9712794	16.4490862	16.9060052	16.5796345
18.0156658	17.2976501	17.4281984	17.7545692	17.8198433	17.8198433


In [17]:
svmclf_model = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('svm', SVC(kernel = 'rbf'))
])
c = [0.1,1,10, 20, 50,100]
gamma = [0.0001,0.001,0.01,0.1,1]
train_errs1 = []
val_errs1 = []
iter = []
best_val_err = float('inf'); best_C = None; best_gamma=None
for C in c:
    for g in gamma:
        iter +=[f"C={C}, g={g}"]
        print(iter[-1])
        svmclf_model.set_params(svm__C=C,svm__gamma=g)
        svmclf_model.fit(x_train_v,y_train_v)
        train_err =100*(1 - svmclf_model.score(x_train_v,y_train_v))
        val_err =100*(1 - svmclf_model.score(x_val,y_val))
        train_errs1 += [train_err]
        val_errs1 +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_C = C
            best_gamma =g
    

C=0.1, g=0.0001


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

C=0.1, g=0.001


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

C=0.1, g=0.01


...

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

C=100, g=0.1


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

C=100, g=1


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [18]:
for i in range(len(train_errs1)):
    print(f"{iter[i]} \n train_err :{train_errs1[i]}\t val_err:{val_errs1[i]}")
    print()
print(f"best C: {best_C}")
print(f"best gamma: {best_gamma}")

C=0.1, g=0.0001 
 train_err :24.579729068059407	 val_err:24.608355091383814

C=0.1, g=0.001 
 train_err :24.579729068059407	 val_err:24.608355091383814

C=0.1, g=0.01 
 train_err :18.508242206626413	 val_err:18.537859007832903

C=0.1, g=0.1 
 train_err :18.051248571894895	 val_err:19.321148825065272

C=0.1, g=1 
 train_err :22.980251346499102	 val_err:23.172323759791126

C=1, g=0.0001 
 train_err :24.579729068059407	 val_err:24.608355091383814

C=1, g=0.001 
 train_err :17.937000163212012	 val_err:18.276762402088774

C=1, g=0.01 
 train_err :16.843479680104455	 val_err:16.97127937336814

C=1, g=0.1 
 train_err :13.97094826179207	 val_err:16.4490861618799

C=1, g=1 
 train_err :7.507752570589199	 val_err:18.146214099216706

C=10, g=0.0001 
 train_err :17.871715358250363	 val_err:18.146214099216706

C=10, g=0.001 
 train_err :16.729231271421575	 val_err:16.90600522193212

C=10, g=0.01 
 train_err :15.652031989554427	 val_err:16.97127937336814

C=10, g=0.1 
 train_err :10.77199281867145	 

In [19]:
for i in range(len(train_errs1)):
    if (((i+1)%5==0 & (i!=0))):
        print(f"{round(train_errs1[i],7)}",end ="\n")
    else:
        print(f"{round(train_errs1[i],7)}",end ="\t")
print()
for i in range(len(val_errs1)):
    if (((i+1)%5==0 & (i!=0))):
        print(f"{round(val_errs1[i],7)}",end ="\n")
    else:
        print(f"{round(val_errs1[i],7)}",end ="\t")

24.5797291	24.5797291	18.5082422	18.0512486	22.9802513
24.5797291	17.9370002	16.8434797	13.9709483	7.5077526
17.8717154	16.7292313	15.652032	10.7719928	1.4036233
17.2515097	16.4680921	15.0644687	9.5805451	0.571242
17.1209401	16.4680921	14.4442631	7.9810674	0.163212
16.9577281	16.8108373	14.0688755	7.0181165	0.081606

24.6083551	24.6083551	18.537859	19.3211488	23.1723238
24.6083551	18.2767624	16.9712794	16.4490862	18.1462141
18.1462141	16.9060052	16.9712794	15.8616188	20.6266319
17.4934726	16.5796345	16.383812	16.383812	21.6057441
17.2976501	16.5796345	16.4490862	17.6240209	22.1932115
16.9712794	16.7101828	16.7101828	18.4725849	22.1279373


In [20]:
#điểm của mô hình mlpclf khi áp dụng tham số tốt nhất dự đoán trên tập tập x_val
mlpclf_model.set_params(mlpclf__alpha=best_alpha,mlpclf__hidden_layer_sizes=best_hls)
mlpclf_model.fit(x_train_v, y_train_v)
y_pred = mlpclf_model.predict(x_val)
print(classification_report(y_val, y_pred))

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


              precision    recall  f1-score   support

      norain       0.87      0.92      0.90      1155
        rain       0.71      0.60      0.65       377

    accuracy                           0.84      1532
   macro avg       0.79      0.76      0.77      1532
weighted avg       0.83      0.84      0.84      1532



C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [21]:
#điểm của mô hình svmclf khi áp dụng tham số tốt nhất dự đoán trên tập tập x_train_v
svmclf_model.set_params(svm__C=best_C,svm__gamma=best_gamma)
svmclf_model.fit(x_train_v,y_train_v)
y_pred = svmclf_model.predict(x_val)

print(classification_report(y_val, y_pred))

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


              precision    recall  f1-score   support

      norain       0.87      0.93      0.90      1155
        rain       0.72      0.58      0.64       377

    accuracy                           0.84      1532
   macro avg       0.80      0.75      0.77      1532
weighted avg       0.83      0.84      0.83      1532



# NHẬN XÉT
- 2 mô hình khá cân băng
- f1 score của SVM của nhãn rain cao hơn của MLP 0.1
- MLP tốt hơn SVM 1 chút trong việc dự đoán ra nhãn "rain" nên chọn MLP

In [22]:
#huấn luyện trên toàn tập train
mlpclf_model.set_params(mlpclf__alpha=best_alpha,mlpclf__hidden_layer_sizes=best_hls)
mlpclf_model.fit(x_train,y_train)
y_pred = svmclf_model.predict(x_train)

print(classification_report(y_train, y_pred))

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


              precision    recall  f1-score   support

      norain       0.89      0.96      0.92      5776
        rain       0.84      0.65      0.73      1883

    accuracy                           0.88      7659
   macro avg       0.86      0.80      0.83      7659
weighted avg       0.88      0.88      0.88      7659



In [23]:
# Điểm mô hình trên test set
y_pred = mlpclf_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      norain       0.89      0.94      0.91       643
        rain       0.78      0.65      0.70       209

    accuracy                           0.87       852
   macro avg       0.83      0.79      0.81       852
weighted avg       0.86      0.87      0.86       852



C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


# Nhận xét
- Độ chính xác của mô mình đạt được mong đợi (>80%)
- Mô hình dự đoán tốt đối với nhãn không mưa (f1 : 0.91)
- Mô hình dự đoán nhãn mưa không tốt lắm, nhưng cũng không hề tệ tuy nhiên độ hiệu quả cũng tương đối
    -Đưa ra được 65% trường hợp mưa với độ chính xác 78%

# Kết luận
- Dự đoán là khi áp dụng vào thực tế, mô hình hoạt động sẽ không tốt, nhưng không quá tệ
- Có thể sử dụng làm nguồn tham khảo nếu không thể xem dự báo thời tiết vì một lý do nào đó 